In [1]:
#%matplotlib widget
import backtrader as bt
import backtrader.analyzers as btanalyzers
import rba_tools as rba
import pandas as pd
import numpy as np
#import matplotlib
from datetime import datetime,date,timedelta
from backtrader.utils import num2date
import rba_tools.backtest.rba_backtrader_set as rbs
import plotly.graph_objects as go

In [2]:
class MaCrossStrategy(bt.Strategy):

    def __init__(self):
        ma_fast = bt.ind.SMA(period = 10)
        ma_slow = bt.ind.SMA(period = 50)

        self.crossover = bt.ind.CrossOver(ma_fast, ma_slow)

    def next(self):
        if not self.position:
            if self.crossover > 0:
                self.buy()
        elif self.crossover < 0:
            self.close()

In [3]:
cerebro = bt.Cerebro()
puller = rba.retriever.get_crypto_data.DataPuller.kraken_puller()
symbol = 'ETH/USD'
from_date = '1/1/20'
to_date = '12/31/20'

In [4]:
dataframe = puller.fetch_df(symbol, 'h', from_date, to_date)
data = bt.feeds.PandasData(dataname=dataframe,
                               # datetime='Date',
                               nocase=True,
                               )
cerebro.adddata(data)

In [5]:
cerebro.addstrategy(MaCrossStrategy)

0

In [6]:
cerebro.broker.setcash(1000.0)

In [7]:
cerebro.addsizer(bt.sizers.PercentSizer, percents = 10)

In [8]:
#cerebro.addanalyzer(btanalyzers.SharpeRatio, _name = 'sharpe')
#cerebro.addanalyzer(btanalyzers.Transactions, _name = 'trans')
#cerebro.addanalyzer(btanalyzers.Returns, _name = 'returns')
cerebro.addanalyzer(btanalyzers.TradeAnalyzer, _name = 'trades') #good for summary stats on trades
cerebro.addanalyzer(btanalyzers.PositionsValue, _name = 'pos', cash=True) #this is a good one. Shows position values over time
cerebro.addanalyzer(btanalyzers.AnnualReturn, _name = 'annReturns')

In [9]:
back = cerebro.run()

In [10]:
cerebro.broker.getvalue()

1225.4636546893448

In [45]:
back[0].crossover.get(ago=0, size=10)

array('d', [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -1.0, 0.0, 0.0, 0.0])

In [47]:
rbs.get_trades_from_cerebro_run(back)

array([        nan,         nan,         nan, ..., 26.26050968,
               nan,         nan])

In [12]:
for x in back[0].analyzers:
    print(x)

In [13]:
mytrades = back[0].analyzers.trades.get_analysis()

In [14]:
def pintdictdict(indict,indent=0):
    indent += 1
    for x in indict:
        print('\n','   ' * indent,x,end='',sep='')
        val = indict[x]
        if isinstance(val,bt.utils.autodict.AutoOrderedDict):
            pintdictdict(val,indent)
        else:
            print('=',val,sep='',end='')

In [15]:
pintdictdict(mytrades)


   total
      total=110
      open=0
      closed=110
   streak
      won
         current=1
         longest=5
      lost
         current=0
         longest=8
   pnl
      gross
         total=225.4636546893422
         average=2.049669588084929
      net
         total=225.4636546893422
         average=2.049669588084929
   won
      total=45
      pnl
         total=365.00202623494334
         average=8.111156138554296
         max=34.667735054043035
   lost
      total=65
      pnl
         total=-139.53837154560117
         average=-2.146744177624633
         max=-8.184438870441925
   long
      total=110
      pnl
         total=225.4636546893422
         average=2.049669588084929
         won
            total=365.00202623494334
            average=8.111156138554296
            max=34.667735054043035
         lost
            total=-139.53837154560117
            average=-2.146744177624633
            max=-8.184438870441925
      won=45
      lost=65
   short
      total=0
  

In [16]:
annReturns = back[0].analyzers.annReturns.get_analysis()
annReturns

OrderedDict([(2020, 0.22546365468934493)])

In [17]:
pos = back[0].analyzers.pos.get_analysis()
for d in [date(2020,1,1) + timedelta(days=x) for x in range(15)]:
    print(d,'=',pos[d])

2020-01-01 = [0.0, 1000.0]
2020-01-02 = [0.0, 1000.0]
2020-01-03 = [101.36157337367626, 899.8714069591528]
2020-01-04 = [101.46747352496219, 899.8714069591528]
2020-01-05 = [102.32223903177008, 899.8714069591528]
2020-01-06 = [109.18305597579428, 899.8714069591528]
2020-01-07 = [108.32072617246598, 899.8714069591528]
2020-01-08 = [0.0, 1005.1739788199698]
2020-01-09 = [0.0, 1005.1739788199698]
2020-01-10 = [103.36323920427873, 904.5067998157474]
2020-01-11 = [101.72277929419151, 904.5067998157474]
2020-01-12 = [104.49016383816475, 904.5067998157474]
2020-01-13 = [0.0, 1006.1011952908887]
2020-01-14 = [113.42013135625615, 905.402354316007]
2020-01-15 = [113.50202807544954, 905.402354316007]


In [18]:
for x in back[0].stats:
    print(x)

In [19]:
datatrades = back[0].stats[1]
broker = back[0].stats[0]

In [20]:
for x in back[0].stats:
    print(type(x))
    print('x.plotinfo.plot =',x.plotinfo.plot)
    print('x.plotinfo.plotskip =',x.plotinfo.plotskip)
    print('x.plotinfo.subplot =',x.plotinfo.subplot)

<class 'backtrader.observers.broker.Broker'>
x.plotinfo.plot = True
x.plotinfo.plotskip = False
x.plotinfo.subplot = True
<class 'backtrader.observers.trades.Trades'>
x.plotinfo.plot = True
x.plotinfo.plotskip = False
x.plotinfo.subplot = True


In [21]:
myobs = back[0].stats[1]
myobs

In [22]:
isinstance(myobs ,bt.observers.trades.DataTrades)

False

In [23]:
ind = myobs

In [24]:
for lineidx in range(ind.size()):
            print('lineidx =',lineidx)
            line = ind.lines[lineidx]
            linealias = ind.lines._getlinealias(lineidx)
            print('linealias =',linealias)
            lineplotinfo = getattr(ind.plotlines, '_%d' % lineidx, None)
            print('ind.plotlines =',ind.plotlines)
            print('lineplotinfo =',lineplotinfo)
            if not lineplotinfo:
                lineplotinfo = getattr(ind.plotlines, linealias, None)
                print('2 lineplotinfo =',lineplotinfo)
            if not lineplotinfo:
                lineplotinfo = AutoInfoClass()
                print('3 lineplotinfo =',lineplotinfo)
            pltmethod = lineplotinfo._get('_method', 'plot')
            print('pltmethod =',pltmethod)
            if pltmethod != 'plot':
                toskip += 1 - lineplotinfo._get('_plotskip', False)

lineidx = 0
linealias = pnlplus
ind.plotlines = <backtrader.metabase.AutoInfoClass_pl_LineSeries_pl_LineIterator_pl_DataAccessor_pl_ObserverBase_pl_Observer_pl_Trades object at 0x00000246FCDFEC10>
lineplotinfo = None
2 lineplotinfo = <backtrader.metabase.AutoInfoClass_pl_Trades_pnlplus object at 0x00000246FCDFED00>
pltmethod = plot
lineidx = 1
linealias = pnlminus
ind.plotlines = <backtrader.metabase.AutoInfoClass_pl_LineSeries_pl_LineIterator_pl_DataAccessor_pl_ObserverBase_pl_Observer_pl_Trades object at 0x00000246FCDFEC10>
lineplotinfo = None
2 lineplotinfo = <backtrader.metabase.AutoInfoClass_pl_Trades_pnlminus object at 0x00000246FCDFED60>
pltmethod = plot


In [25]:
for line in myobs.lines:
    npary = np.frombuffer(line.array)
    print(len(npary))
    print(npary[~np.isnan(npary)])
    print(len(npary[~np.isnan(npary)]))

17564
[ 5.17397882  0.92721647  9.09053583  0.48992148  0.65139018  8.057664
  5.70578469 17.48717382 15.42049358  0.82393119  1.65650065  0.05705387
 16.06996126  4.47494635  0.54166023  2.97942328 18.48871935  5.00937118
 10.31742777  6.40132012  2.15891017  4.47967195  4.67630929 13.24868796
  3.41727298  0.80775984  5.75404907  0.99245125  0.51938014 34.66773505
 19.9267267  12.9877957  12.636903    3.27049686  1.07265577  3.03225219
  5.35753484  9.69601655 10.01619945  2.2411854  22.72046183 18.96825994
  6.14210989 10.1262166  26.26050968]
45
17564
[-2.44184724 -3.78800498 -1.25815945 -3.39461277 -3.38213022 -1.11740228
 -1.6173047  -8.18443887 -4.21430947 -2.24797612 -2.36340082 -4.52201808
 -1.19268459 -5.23053695 -0.94006072 -0.55499168 -2.33104183 -3.02447357
 -3.92262669 -2.08682787 -0.0463256  -0.72412152 -2.17517885 -0.34960226
 -0.68782093 -3.00860097 -0.08313186 -0.78545996 -0.05341116 -0.9855769
 -2.54855387 -1.49995636 -1.53366128 -1.32915585 -0.26674974 -0.44422128
 

In [26]:
npary = np.frombuffer(myobs.lines[0].array)
npary

array([nan, nan, nan, ..., nan, nan, nan])

In [27]:
npary[~np.isnan(npary)]

array([ 5.17397882,  0.92721647,  9.09053583,  0.48992148,  0.65139018,
        8.057664  ,  5.70578469, 17.48717382, 15.42049358,  0.82393119,
        1.65650065,  0.05705387, 16.06996126,  4.47494635,  0.54166023,
        2.97942328, 18.48871935,  5.00937118, 10.31742777,  6.40132012,
        2.15891017,  4.47967195,  4.67630929, 13.24868796,  3.41727298,
        0.80775984,  5.75404907,  0.99245125,  0.51938014, 34.66773505,
       19.9267267 , 12.9877957 , 12.636903  ,  3.27049686,  1.07265577,
        3.03225219,  5.35753484,  9.69601655, 10.01619945,  2.2411854 ,
       22.72046183, 18.96825994,  6.14210989, 10.1262166 , 26.26050968])

In [28]:
myline = myobs.lines[0]
myline

In [29]:
myline.array[0:1000]

array('d', [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 5.173978819969746, nan, nan, nan, nan, nan, nan, nan, nan, nan,

In [30]:
data = {'trades': myline.array}
df = pd.DataFrame(data=data)
df

,trades
0,NaN
1,NaN
2,NaN
3,NaN
4,NaN
...,...
17559,NaN
17560,NaN
17561,NaN
17562,NaN


In [31]:
df.loc[np.isnan(df['trades']) == False]

,trades
185,5.173979
296,0.927216
370,9.090536
447,0.489921
521,0.651390
702,8.057664
821,5.705785
967,17.487174
1101,15.420494
1314,0.823931


In [32]:
def get_trades_from_cerebro_run(cerebro_run):
    #get trades. Long is 'long' and shorts is 'short'. This is unconfirmed. Need to verify
    array_list = []
    for strat in cerebro_run[0].stats:
        if isinstance(strat, bt.observers.trades.Trades):
            for line in strat.lines:
                pnl_data = np.frombuffer(line.array)
                np.nan_to_num(pnl_data, copy=False, nan=0)
                array_list.append(pnl_data)
    summed_ary = sum(array_list)
    summed_ary[summed_ary == 0] = np.nan
    return summed_ary[:int(len(summed_ary) / 2)] #not sure what the deal is with the double sized array but this seems to work
result = get_trades_from_cerebro_run(back)
print(len(result))
print(result[~np.isnan(result)])

8782
[ 5.17397882  0.92721647  9.09053583  0.48992148  0.65139018 -2.44184724
  8.057664    5.70578469 17.48717382 15.42049358 -3.78800498 -1.25815945
  0.82393119 -3.39461277 -3.38213022 -1.11740228  1.65650065  0.05705387
 -1.6173047  -8.18443887 -4.21430947 16.06996126 -2.24797612  4.47494635
 -2.36340082 -4.52201808  0.54166023  2.97942328 18.48871935 -1.19268459
 -5.23053695 -0.94006072  5.00937118 10.31742777  6.40132012 -0.55499168
 -2.33104183 -3.02447357 -3.92262669  2.15891017  4.47967195  4.67630929
 -2.08682787 13.24868796 -0.0463256  -0.72412152 -2.17517885 -0.34960226
 -0.68782093 -3.00860097 -0.08313186 -0.78545996  3.41727298  0.80775984
 -0.05341116 -0.9855769  -2.54855387  5.75404907 -1.49995636 -1.53366128
  0.99245125  0.51938014 34.66773505 -1.32915585 19.9267267  -0.26674974
 -0.44422128 -0.0730787  12.9877957  -3.40324277 -4.13483837 -1.22286196
 -3.3169801  12.636903   -6.82372571 -5.23904855  3.27049686 -3.37980343
  1.07265577 -0.79558336  3.03225219 -1.925965

In [33]:
count = 0
for x in result:
    if not np.isnan(x):
        print('count =',count,'val =',x)
    count +=1

count = 185 val = 5.173978819969746
count = 296 val = 0.9272164709188763
count = 370 val = 9.090535830467122
count = 447 val = 0.48992148294800975
count = 521 val = 0.6513901751897789
count = 580 val = -2.441847239319047
count = 702 val = 8.057664000375246
count = 821 val = 5.705784685769992
count = 967 val = 17.487173822075007
count = 1101 val = 15.420493582712849
count = 1200 val = -3.7880049755521568
count = 1261 val = -1.2581594488100298
count = 1314 val = 0.8239311940346661
count = 1402 val = -3.394612770706136
count = 1431 val = -3.3821302173278713
count = 1509 val = -1.1174022769996212
count = 1610 val = 1.6565006450556075
count = 1679 val = 0.0570538712416639
count = 1688 val = -1.6173047048099023
count = 1777 val = -8.184438870441925
count = 1861 val = -4.214309469865906
count = 1923 val = 16.069961259417344
count = 1957 val = -2.247976124237581
count = 2034 val = 4.4749463524091535
count = 2080 val = -2.363400820556676
count = 2088 val = -4.52201808346843
count = 2193 val = 0

In [34]:
result[:int(len(result) / 2)]

array([nan, nan, nan, ..., nan, nan, nan])

In [35]:
def get_buy_sell_from_cerebro_run(cerebro_run, trade_type='buy'):
    #get executed buy or sell orders. type can switch between buy and sell
    data = None
    for strat in cerebro_run[0].getobservers():
        if isinstance(strat, bt.observers.buysell.BuySell):
            line = 0 if trade_type == 'buy' else 1
            data = np.frombuffer(strat.lines[line].array)
    return data[:int(len(data) / 2)] #not sure what the deal is with the double sized array but this seems to work
result = get_buy_sell_from_cerebro_run(back)
print(len(result))
print(result[~np.isnan(result)])

8782
[129.823   138.65845 144.43055 160.93915 163.97295 159.3927  157.6
 172.2765  187.53415 229.0125  259.8627  259.51795 256.22805 218.67
 226.67805 219.3201  225.0725  199.05865 197.7289  124.1297  114.2206
 114.72295 124.53355 128.4046  136.57025 134.60025 129.53735 133.93045
 141.31795 169.026   158.3289  155.38375 165.37165 172.46365 193.2767
 206.5151  209.3913  205.50055 203.64875 205.38235 186.7166  197.197
 203.4222  202.8706  232.09555 239.89675 238.63595 240.0051  239.24665
 240.90145 231.5144  228.68745 226.75685 220.3642  221.7235  223.84125
 225.1316  224.4421  237.02055 237.8972  229.67245 231.672   233.9966
 317.17    325.7395  384.86905 389.87285 383.19455 380.5646  433.0848
 408.23325 386.01165 375.088   381.50035 343.6665  346.04035 341.007
 367.8975  372.73385 377.7475  338.9582  351.95035 351.65485 342.19885
 344.7106  377.0777  376.02375 371.74885 363.1695  370.6949  403.60375
 396.43295 380.12135 377.71795 394.42355 442.33395 455.5625  453.0015
 453.9274  475.78

In [36]:
x = 0 if True else 1
print(x)

0


In [37]:
def get_broker_cash_from_cerebro_run(cerebro_run):
    data = None
    for strat in cerebro_run[0].stats:
        if isinstance(strat, bt.observers.broker.Broker):
            #assumes we want line 0 which I think is safe for Broker?
            npary = np.frombuffer(strat.lines[0].array)
            data = npary
    return data
result = get_broker_cash_from_cerebro_run(back)
print(result)
print(len(result))

[1000. 1000. 1000. ...   nan   nan   nan]
17564


In [38]:
def get_pos_analysis(cerebro_run):
    for analyzer in cerebro_run[0].analyzers:
        if isinstance(analyzer, btanalyzers.PositionsValue):
            return analyzer.get_analysis().copy()

In [39]:
def get_cash_including_position(cerebro_run):
    #this assumes that PositionsValue analyzer exists with parameter cash=True (this is not the default) and headers=False (this is the default)
    pos_analysis = get_pos_analysis(cerebro_run)
    data = []
    for key, value in pos_analysis.items():
        data.append(sum(value))
    return np.round_(np.array(data),2)
result = get_cash_including_position(back)
result

array([1000.  , 1000.  , 1001.23, 1001.34, 1002.19, 1009.05, 1008.19,
       1005.17, 1005.17, 1007.87, 1006.23, 1009.  , 1006.1 , 1018.82,
       1018.9 , 1015.19, 1018.93, 1021.68, 1015.68, 1015.68, 1017.13,
       1016.33, 1016.33, 1016.1 , 1013.51, 1018.1 , 1019.67, 1023.78,
       1021.94, 1026.35, 1023.65, 1025.81, 1028.61, 1029.49, 1027.65,
       1034.72, 1039.66, 1045.05, 1045.24, 1048.14, 1045.14, 1047.48,
       1060.32, 1061.33, 1069.12, 1060.56, 1060.56, 1060.56, 1066.35,
       1056.87, 1056.77, 1056.79, 1055.7 , 1061.02, 1056.34, 1056.34,
       1056.34, 1056.76, 1052.95, 1049.56, 1049.56, 1052.85, 1048.45,
       1048.45, 1048.2 , 1056.01, 1052.29, 1050.1 , 1050.1 , 1050.16,
       1048.54, 1048.54, 1048.54, 1040.19, 1040.36, 1040.36, 1039.43,
       1037.97, 1054.1 , 1051.21, 1052.21, 1049.97, 1054.83, 1056.96,
       1054.44, 1054.44, 1047.55, 1047.55, 1047.55, 1048.17, 1048.82,
       1048.1 , 1051.96, 1051.84, 1051.42, 1050.31, 1071.3 , 1066.16,
       1070.64, 1068

In [40]:
def get_percent_cash_change(cerebro_run):
    cash_vals = get_cash_including_position(cerebro_run)
    start_val = cash_vals[0]
    return np.round((cash_vals / start_val - 1) * 100,2)
result = get_percent_cash_change(back)
print(result)

[ 0.    0.    0.12  0.13  0.22  0.91  0.82  0.52  0.52  0.79  0.62  0.9
  0.61  1.88  1.89  1.52  1.89  2.17  1.57  1.57  1.71  1.63  1.63  1.61
  1.35  1.81  1.97  2.38  2.19  2.63  2.36  2.58  2.86  2.95  2.77  3.47
  3.97  4.51  4.52  4.81  4.51  4.75  6.03  6.13  6.91  6.06  6.06  6.06
  6.63  5.69  5.68  5.68  5.57  6.1   5.63  5.63  5.63  5.68  5.3   4.96
  4.96  5.28  4.85  4.85  4.82  5.6   5.23  5.01  5.01  5.02  4.85  4.85
  4.85  4.02  4.04  4.04  3.94  3.8   5.41  5.12  5.22  5.    5.48  5.7
  5.44  5.44  4.75  4.75  4.75  4.82  4.88  4.81  5.2   5.18  5.14  5.03
  7.13  6.62  7.06  6.82  6.84  6.84  6.7   6.31  6.34  6.22  6.52  6.41
  7.46  6.99  6.72  6.72  7.21  7.36  7.49  7.9   8.1   7.75  7.71  8.73
  8.39  8.34  8.4   8.1   8.1   7.8   7.42  7.61  7.57  7.63  7.63  7.63
  7.62  8.2   8.39  8.07  8.09  8.44  8.87  8.86  8.54  8.54  8.5   8.47
  8.33  8.33  8.33  8.44  9.07  9.09 10.3   9.67 10.2   9.65  9.65  9.62
  9.58  9.58  9.36  9.47  9.29  9.37  8.96  8.96  9.0

In [41]:
result

array([ 0.  ,  0.  ,  0.12,  0.13,  0.22,  0.91,  0.82,  0.52,  0.52,
        0.79,  0.62,  0.9 ,  0.61,  1.88,  1.89,  1.52,  1.89,  2.17,
        1.57,  1.57,  1.71,  1.63,  1.63,  1.61,  1.35,  1.81,  1.97,
        2.38,  2.19,  2.63,  2.36,  2.58,  2.86,  2.95,  2.77,  3.47,
        3.97,  4.51,  4.52,  4.81,  4.51,  4.75,  6.03,  6.13,  6.91,
        6.06,  6.06,  6.06,  6.63,  5.69,  5.68,  5.68,  5.57,  6.1 ,
        5.63,  5.63,  5.63,  5.68,  5.3 ,  4.96,  4.96,  5.28,  4.85,
        4.85,  4.82,  5.6 ,  5.23,  5.01,  5.01,  5.02,  4.85,  4.85,
        4.85,  4.02,  4.04,  4.04,  3.94,  3.8 ,  5.41,  5.12,  5.22,
        5.  ,  5.48,  5.7 ,  5.44,  5.44,  4.75,  4.75,  4.75,  4.82,
        4.88,  4.81,  5.2 ,  5.18,  5.14,  5.03,  7.13,  6.62,  7.06,
        6.82,  6.84,  6.84,  6.7 ,  6.31,  6.34,  6.22,  6.52,  6.41,
        7.46,  6.99,  6.72,  6.72,  7.21,  7.36,  7.49,  7.9 ,  8.1 ,
        7.75,  7.71,  8.73,  8.39,  8.34,  8.4 ,  8.1 ,  8.1 ,  7.8 ,
        7.42,  7.61,

In [42]:
rbs.get_datetime_array(back)[0:10]

array([datetime.datetime(2020, 1, 1, 0, 0),
       datetime.datetime(2020, 1, 1, 1, 0),
       datetime.datetime(2020, 1, 1, 2, 0),
       datetime.datetime(2020, 1, 1, 3, 0),
       datetime.datetime(2020, 1, 1, 4, 0),
       datetime.datetime(2020, 1, 1, 5, 0),
       datetime.datetime(2020, 1, 1, 6, 0),
       datetime.datetime(2020, 1, 1, 7, 0),
       datetime.datetime(2020, 1, 1, 8, 0),
       datetime.datetime(2020, 1, 1, 9, 0)], dtype=object)

In [43]:
def get_ohlcv_data_from_cerebro_run(cerebro_run):
    data = cerebro_run[0].datas[0]
    start = 0
    end = len(data)
    return pd.DataFrame({
        'Open' : data.open.plotrange(start,end),
        'High' : data.high.plotrange(start,end),
        'Low' : data.low.plotrange(start,end),
        'Close' : data.close.plotrange(start,end),
        'Volume' : data.volume.plotrange(start,end)
        })
val = get_ohlcv_data_from_cerebro_run(back)
val

,Open,High,Low,Close,Volume
0,128.66,128.66,128.19,128.42,515.780816
1,128.33,130.05,128.30,130.05,555.054287
2,130.20,130.58,130.05,130.44,582.304327
3,130.48,130.48,129.61,129.63,480.119100
4,129.73,130.13,129.67,129.67,239.555071
...,...,...,...,...,...
8777,740.83,742.97,738.44,739.53,1559.446486
8778,739.46,745.37,739.07,745.09,1365.984380
8779,745.38,745.68,737.49,739.08,1334.269917
8780,739.27,743.00,736.68,741.50,1388.991292


In [44]:
def summarize_cerebro_run(cerebro_run):
    #get pandas dataframe of summarized data
    index = rbs.get_datetime_array(cerebro_run)
    ohlcv_df = get_ohlcv_data_from_cerebro_run(cerebro_run)
    data = { 'Open' :ohlcv_df['Open'].values,
             'High' : ohlcv_df['High'].values,
             'Low' : ohlcv_df['Low'].values,
             'Close' : ohlcv_df['Close'].values,
             'Volume' : ohlcv_df['Volume'].values,
             'cash' : rbs.get_cash_including_position(cerebro_run),
             'percent_change' : rbs.get_percent_cash_change(cerebro_run),
             'trades' : rbs.get_trades_from_cerebro_run(cerebro_run),
             'buy' : get_buy_sell_from_cerebro_run(cerebro_run, trade_type='buy'),
             'sell' : get_buy_sell_from_cerebro_run(cerebro_run, trade_type='sell')
            }
    return pd.DataFrame(data=data, index=index)
df = summarize_cerebro_run(back)
df

ValueError: could not broadcast input array from shape (366) into shape (8782)

In [ ]:
df['trades']

2018-01-01   NaN
2018-01-02   NaN
2018-01-03   NaN
2018-01-04   NaN
2018-01-05   NaN
              ..
2019-12-27   NaN
2019-12-28   NaN
2019-12-29   NaN
2019-12-30   NaN
2019-12-31   NaN
Name: trades, Length: 730, dtype: float64

In [ ]:
df.loc['2018-06-01':'2018-06-30',:]

,Open,High,Low,Close,Volume,cash,percent_change,trades,buy,sell
2018-06-01,0.077218,0.077750,0.075917,0.076980,122731.661,1013.47,1.35,NaN,NaN,NaN
2018-06-02,0.076980,0.078121,0.076510,0.077350,113457.714,1014.01,1.40,NaN,NaN,NaN
2018-06-03,0.077375,0.080857,0.077350,0.080234,109639.476,1018.26,1.83,NaN,NaN,NaN
2018-06-04,0.080299,0.080799,0.077735,0.078925,120832.454,1016.33,1.63,NaN,NaN,NaN
2018-06-05,0.079001,0.080500,0.077695,0.079898,120506.520,1017.76,1.78,NaN,NaN,NaN
2018-06-06,0.079898,0.080100,0.079001,0.079235,109281.535,1017.76,1.78,17.762267,NaN,0.081301
2018-06-07,0.079232,0.079700,0.078230,0.078698,107771.151,1017.76,1.78,NaN,NaN,NaN
2018-06-08,0.078696,0.079289,0.077991,0.078751,99569.817,1017.83,1.78,NaN,0.076821,NaN
2018-06-09,0.078771,0.079500,0.077700,0.079108,84598.599,1018.30,1.83,NaN,NaN,NaN
2018-06-10,0.079108,0.079500,0.075500,0.077540,129509.836,1016.27,1.63,NaN,NaN,NaN


In [ ]:
df = rbs.summarize_cerebro_run(back)
df

,Open,High,Low,Close,Volume,cash,percent_change,trades,buy,sell
2018-01-01,0.053586,0.057200,0.053401,0.056367,312440.757,1000.0,0.00,NaN,NaN,NaN
2018-01-02,0.056368,0.066265,0.051040,0.058261,492006.759,1000.0,0.00,NaN,NaN,NaN
2018-01-03,0.058300,0.064210,0.056449,0.062600,469486.617,1000.0,0.00,NaN,NaN,NaN
2018-01-04,0.062600,0.070000,0.060125,0.062236,595991.613,1000.0,0.00,NaN,NaN,NaN
2018-01-05,0.062449,0.069758,0.054000,0.056584,710527.221,1000.0,0.00,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
2019-12-27,0.017438,0.017525,0.017258,0.017415,109292.550,971.1,-2.89,NaN,NaN,NaN
2019-12-28,0.017409,0.017647,0.017370,0.017512,93429.335,971.1,-2.89,NaN,NaN,NaN
2019-12-29,0.017513,0.018336,0.017484,0.018184,136887.525,971.1,-2.89,NaN,NaN,NaN
2019-12-30,0.018185,0.018402,0.017916,0.018161,134313.801,971.1,-2.89,NaN,NaN,NaN


In [ ]:
myplot = rbs.get_candlestick_plot(df)

In [ ]:
myfig = go.Figure([myplot])
myfig.show()

In [ ]:
def plotstrategy(cerebro_run_strategy):
    pass

In [ ]:
import backtrader.plot.plot as plot

In [ ]:
plotter = plot.Plot()

In [ ]:
plotlist = []
numfigs=1
iplot=False
start=None
end=None
use=None
for stratlist in cerebro.runstrats:
    for si, strat in enumerate(stratlist):
        print(f"si={si}\nstrat={strat}")
        rfig = plotter.plot(strat, figid=si * 100,
                                    numfigs=numfigs, iplot=iplot,
                                   start=start, end=end, use=use)
        plotlist.append(rfig)
plotlist

si=0
strat=<__main__.MaCrossStrategy object at 0x000001C87A93DDC0>


[[<Figure size 640x480 with 5 Axes>]]

In [ ]:
fig = plotlist[0][0]
fig

<Figure size 640x480 with 5 Axes>

In [ ]:
indent = "   "
for axis in fig.axes:
    print("axis",axis)
    for line in axis.lines:
        print(indent,"line:",line)
    print(indent,"axis.axes:",axis.axes)
    for line in axis.axes.lines:
        print(indent*2,"line:",line)

axis AxesSubplot(0.05,0.86375;0.9x0.11625)
    line: Line2D(Broker (None)
cash 971.10)
    line: Line2D(value 971.10)
    line: Line2D(_line2)
    line: Line2D(_line3)
    axis.axes: AxesSubplot(0.05,0.86375;0.9x0.11625)
       line: Line2D(Broker (None)
cash 971.10)
       line: Line2D(value 971.10)
       line: Line2D(_line2)
       line: Line2D(_line3)
axis AxesSubplot(0.05,0.7475;0.9x0.11625)
    line: Line2D(Trades - Net Profit/Loss (True)
Positive 0.00)
    line: Line2D(Negative 0.00)
    line: Line2D(_line2)
    line: Line2D(_line3)
    line: Line2D(_line4)
    axis.axes: AxesSubplot(0.05,0.7475;0.9x0.11625)
       line: Line2D(Trades - Net Profit/Loss (True)
Positive 0.00)
       line: Line2D(Negative 0.00)
       line: Line2D(_line2)
       line: Line2D(_line3)
       line: Line2D(_line4)
axis AxesSubplot(0.05,0.16625;0.9x0.58125)
    line: Line2D(_line0)
    line: Line2D(_line1)
    axis.axes: AxesSubplot(0.05,0.16625;0.9x0.58125)
       line: Line2D(_line0)
       line: Line